In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [2]:
train = pd.read_csv("/Users/mac/Documents/MySchoolDocs/PERSONAL/JOB APPLICATIONS/ENERGY_PROJECT/data/final_datasets/train_df.csv")
eval = pd.read_csv("/Users/mac/Documents/MySchoolDocs/PERSONAL/JOB APPLICATIONS/ENERGY_PROJECT/data/final_datasets/eval_df.csv")
holdout = pd.read_csv("/Users/mac/Documents/MySchoolDocs/PERSONAL/JOB APPLICATIONS/ENERGY_PROJECT/data/final_datasets/holdout_df.csv")

In [3]:
X_train = train.drop(columns=["Date", "Strain_Index", "target_strain_index"])
y_train = train["target_strain_index"]

X_test = eval.drop(columns=["Date", "Strain_Index", "target_strain_index"])
y_test = eval["target_strain_index"]

X_valid = holdout.drop(columns=["Date", "Strain_Index", "target_strain_index"])
y_valid = holdout["target_strain_index"]

In [4]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

In [5]:
xgb_model = xgb.XGBRegressor(
    objective="reg:squarederror",
    n_estimators=5000,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_lambda=1.0,
    reg_alpha=0.2,
    random_state=42,
    verbosity=0
)

In [6]:
xgb_model.fit(
    X_train, y_train
)

y_pred_xgb = xgb_model.predict(X_test)

In [7]:
def mape(y_true, y_pred, eps=1e-6):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return np.mean(np.abs((y_true - y_pred) / np.clip(np.abs(y_true), eps, None))) * 100

def smape(y_true, y_pred, eps=1e-6):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) /
                         (np.abs(y_true) + np.abs(y_pred) + eps))


print("XGBoost on Strain Index (t+1)")
print("--------------------------------")
print(f"MAE  : {mean_absolute_error(y_test, y_pred_xgb):.4f}")
print(f"RMSE : {root_mean_squared_error(y_test, y_pred_xgb):.4f}")
print(f"MAPE : {mape(y_test, y_pred_xgb):.2f}%")
print(f"SMAPE: {smape(y_test, y_pred_xgb):.2f}%")

XGBoost on Strain Index (t+1)
--------------------------------
MAE  : 0.0349
RMSE : 0.0406
MAPE : 42.98%
SMAPE: 32.85%
